In [1]:
import nltk
#nltk.download()

In [2]:
import requests
import pandas as pd
import re, string, unicodedata
from pprint import pprint
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import FreqDist
from textblob import TextBlob

#import contractions
#import inflect

In [3]:
violence_words = set(["asesinato","asesino","asesina","matar","mata","mato", "apuñalar","apuñalo","pega","pego","apalizo","golpea","golpeo","agredio","agrede"])

In [4]:
print(violence_words)

{'apuñalar', 'agrede', 'agredio', 'apalizo', 'asesina', 'pega', 'golpea', 'mato', 'apuñalo', 'asesinato', 'asesino', 'golpeo', 'mata', 'matar', 'pego'}


In [5]:
stopWords = set(stopwords.words('spanish'))

In [6]:
print(stopWords)

{'tengas', 'tendré', 'ya', 'los', 'algunos', 'nada', 'ellas', 'nos', 'fueron', 'habrían', 'fueras', 'estuviera', 'tendría', 'habíamos', 'hayas', 'el', 'seáis', 'estas', 'estuvieras', 'estábamos', 'tuvisteis', 'hubieron', 'hubiésemos', 'tuvieran', 'donde', 'tendríais', 'otro', 'las', 'soy', 'ha', 'sin', 'hemos', 'sus', 'un', 'había', 'suyo', 'fueseis', 'tuve', 'la', 'estuvieron', 'he', 'tiene', 'tuyas', 'uno', 'les', 'más', 'habréis', 'estaríamos', 'habías', 'vosostros', 'fuese', 'hasta', 'cual', 'sea', 'estaréis', 'estaría', 'míos', 'mío', 'sintiendo', 'éramos', 'contra', 'fuisteis', 'estoy', 'tendréis', 'tú', 'sentidos', 'has', 'tuvieras', 'tenemos', 'es', 'tengamos', 'tienes', 'estada', 'a', 'que', 'esas', 'sean', 'erais', 'fuesen', 'habrán', 'lo', 'en', 'antes', 'hubieran', 'estadas', 'tendríamos', 'hayan', 'sois', 'serás', 'tendrías', 'teníamos', 'nuestra', 'tuviese', 'tenían', 'habéis', 'tendrás', 'para', 'mía', 'habrás', 'fueran', 'estarías', 'haya', 'mucho', 'os', 'algo', 'estuv

In [7]:
MADRID_URL = "https://www.madridiario.es/sucesos"

In [8]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser") #parser para que no se queje
    return soup

soup = get_soup(MADRID_URL)

In [9]:
def get_headers(soup):
    lst = []
    total_lst = []
    
    #esto el container de cada noticia
    #headers_container = soup.find_all("div",{'class': 'n1 fueraNoticia'})
    
    
    #este código funciona pero sólo los headers
    headers_container = soup.find_all("h2", {'class': 'titulo'})
    for index, header in enumerate(headers_container):
        lst.append(header.text)
    
    return lst

    
    """
    headers_container = soup.find_all("meta", {'itemprop': 'datePublished'})
    lst = [item for index, item in enumerate(headers_container)]
    lst = [item.text.replace('<meta content="','').replace('itemprop="datePublished"/>','') for item in lst] 
        
    return lst

    """
    

    #items = [lines.find('h3', {'class','title'}) for lines in headers_container]    

    """
 depth = line.find('td',{'class','tabev3' })    
    
    #es una prueba para buscar el título, la fecha, y un hueco para el boleano
    for item in headers_container:
        #lst.append(item.find("h2", {'class': 'titulo'}))
        lst.append(item.find("meta", {'itemprop','datePublished'}))
        lst.append([])
        total_lst.append(lst)
        return lst
    """
    

news_raw = get_headers(soup)


In [10]:
# Get rid of lower-cases
def tolower (work_list):
    return [row.lower() for row in work_list]
    
news_raw = tolower(news_raw)
pprint(news_raw[:6])

['una anciana, crítica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermería de alcalá, culpable de uno de los asesinatos',
 'desarticulada una organización de trata de transexuales',
 'detenidos por acordar el asesinato de un hombre para vender sus órganos',
 'libertad para el presunto pederasta que simulaba ser agente secreto',
 'agrede a su pareja en hortaleza mordiéndole el labio']


In [11]:
# Get rid of accents
def clean_accent(work_list):
    return [row.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u') for row in work_list]

news_raw2 = clean_accent(news_raw)
pprint(news_raw2[:6])

['una anciana, critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala, culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales',
 'detenidos por acordar el asesinato de un hombre para vender sus organos',
 'libertad para el presunto pederasta que simulaba ser agente secreto',
 'agrede a su pareja en hortaleza mordiendole el labio']


In [12]:
#Get rid of commas
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw3 = clean_any_character(news_raw2, ',')
pprint(news_raw3[:3])

['una anciana critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales']


In [13]:
#Get rid of quotes
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw4 = clean_any_character(news_raw3, '\"')
pprint(news_raw4[:4])

['una anciana critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales',
 'detenidos por acordar el asesinato de un hombre para vender sus organos']


In [14]:
#Get rid of '
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw5 = clean_any_character(news_raw4, '\'')
pprint(news_raw5[:4])

['una anciana critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales',
 'detenidos por acordar el asesinato de un hombre para vender sus organos']


In [15]:
#Get rid of '
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw6 = clean_any_character(news_raw5, ':')
pprint(news_raw6[:4])

['una anciana critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales',
 'detenidos por acordar el asesinato de un hombre para vender sus organos']


In [16]:
# Get each word / Tokenizar
def token(work_list):
    return [nltk.word_tokenize(row) for row in work_list]

token_news = token(news_raw6)
print(token_news[:6])

[['una', 'anciana', 'critica', 'al', 'quedar', 'atrapada', 'en', 'su', 'coche', 'accidentado'], ['la', 'auxiliar', 'de', 'enfermeria', 'de', 'alcala', 'culpable', 'de', 'uno', 'de', 'los', 'asesinatos'], ['desarticulada', 'una', 'organizacion', 'de', 'trata', 'de', 'transexuales'], ['detenidos', 'por', 'acordar', 'el', 'asesinato', 'de', 'un', 'hombre', 'para', 'vender', 'sus', 'organos'], ['libertad', 'para', 'el', 'presunto', 'pederasta', 'que', 'simulaba', 'ser', 'agente', 'secreto'], ['agrede', 'a', 'su', 'pareja', 'en', 'hortaleza', 'mordiendole', 'el', 'labio']]


In [17]:
def clean_stopwords(list_tokens, stp_words):
    [row.remove(item) for row in list_tokens for item in row if item in stp_words]
    return list_tokens
    
cleaned_data = clean_stopwords(token_news, stopWords)
pprint(cleaned_data[:6])

[['anciana', 'critica', 'quedar', 'atrapada', 'su', 'coche', 'accidentado'],
 ['auxiliar', 'enfermeria', 'alcala', 'culpable', 'uno', 'los', 'asesinatos'],
 ['desarticulada', 'organizacion', 'trata', 'transexuales'],
 ['detenidos', 'acordar', 'asesinato', 'un', 'hombre', 'vender', 'organos'],
 ['libertad',
  'el',
  'presunto',
  'pederasta',
  'simulaba',
  'ser',
  'agente',
  'secreto'],
 ['agrede', 'su', 'pareja', 'hortaleza', 'mordiendole', 'labio']]


In [28]:


#new = new + " enfado"
new = [' '.join(item) for item in cleaned_data]


for item in new:
    if TextBlob(item).detect_language() == 'es':
        print("1")
        analysis = TextBlob(str(item))
        item = analysis.translate(to='spanish')
        print(item)
    analysis = TextBlob(str(item))
    print(analysis.sentiment,"\n")




"""
new = ' '.join(cleaned_data[1])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")


new = ' '.join(cleaned_data[2])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")

"""
"""
new2 = analysis.translate(to='spanish')
analysis = TextBlob(str(new2))
print(analysis.sentiment)
print(analysis.tags)
#print(analysis.translate(to='spanish'))
print(dir(analysis))
"""


1
old lady criticizes getting caught her car accident
Sentiment(polarity=0.1, subjectivity=0.2) 

1
auxiliary nursing alcala guilty one the murders
Sentiment(polarity=-0.5, subjectivity=1.0) 

1
disjointed organization treats transsexuals
Sentiment(polarity=0.0, subjectivity=0.0) 

1
detainees accord murder a man sell organs
Sentiment(polarity=0.0, subjectivity=0.0) 

1
freedom the alleged pedophile pretended to be a secret agent
Sentiment(polarity=-0.25, subjectivity=0.39999999999999997) 

1
assaults her partner hortaleza biting her lip
Sentiment(polarity=0.0, subjectivity=0.0) 

1
king the burundanga arrested new
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453) 

1
arrested pederasta pretended to be secret agent
Sentiment(polarity=-0.4, subjectivity=0.7) 

1
dies worker vallecas rush the elevator shaft
Sentiment(polarity=0.0, subjectivity=0.0) 

1
suffocated fire the palace chanting peak
Sentiment(polarity=0.0, subjectivity=0.0) 

1
burns shop nuts improved f

"\nnew2 = analysis.translate(to='spanish')\nanalysis = TextBlob(str(new2))\nprint(analysis.sentiment)\nprint(analysis.tags)\n#print(analysis.translate(to='spanish'))\nprint(dir(analysis))\n"

In [ ]:
def new_item(list_tokens):
    for index, row in enumerate(list_tokens):
        list_tokens.insert(index+1, [])
        list_tokens.insert(index+1, [])
    return list_tokens
 
print(new_item(cleaned_data))

In [ ]:
def violence_at_row(list_tokens,set_list):
    for row in list_tokens:
        for word in row:
            if word in violence_words:
                

In [ ]:
for word in cleaned_data:
    if 